Data: 
Option prices and implied volatility from post-no-preference option chain dataset. data spans from 2019-current. collected more recently on mon-wed-fri. Many but not all options are posted there for each security.

AAPL price data from Kaggle(?)

Treasury Bond rates from home.treasury.gov. Daily treasury par yield rates

In [1]:
import pandas as pd
import numpy as np
#10m rows takes about 30 seconds... Expect long processing times for full data. do in batches.
#f10m = pd.read_csv('pnp_options.csv', nrows=10000000)

f1m = pd.read_csv('pnp_options.csv', usecols=['date', 'act_symbol', 'expiration', 'strike', 'call_put', 'bid', 'ask',
       'vol'] ,nrows = 1000000)

prices = pd.read_csv('HistoricalQuotes.csv')

rates = pd.read_csv('treasury_2019.csv', usecols=['Date', '1 Mo', '2 Mo', '3 Mo', '6 Mo', '1 Yr'])


### NOTE...

**The following data engineering assumes**:

1. we are only interested in AAPL options. if we want to expand, we can simply modify the first line to include more act_symbol values

2. our prices dataframe includes prices covering the entire range of dates for option prices, plus an extra n_prices before the earliest option price, such that we can utilize the n_prices preceding the option pricing in our LSTM model down the line 

3. our rates dataframe also contains treasury rates with dates covering all issuance dates of options.
    
4. all of our option time-to-expiries are closest to 1-3 months, not 6+ months. in the 1m row data the longest time-to-expiry is 62 days. if we see in the full data a time-to-expiry longer than 135 days, we need to add an option to use the 6 month treasury rate in our determine_r function.


**Also**, you will probably want to split this cell into multiple smaller ones when we're working with the full data. some of the actions might be computationally expensive.

In [2]:
df = f1m[f1m['act_symbol'] == 'AAPL']
aapl_prices = prices.copy()
rate = rates.copy()

#remove expiration date, replace with int # of days until expiration
df['days_expiry'] = (pd.to_datetime(df['expiration']) - pd.to_datetime(df['date'])).dt.days
df = df.drop(['expiration'], axis=1)

#format dates on df, aapl_prices, rates to match each other

df['date'] = pd.to_numeric(df['date'].str.replace('-',''))
aapl_prices['Date'] = pd.to_datetime(aapl_prices['Date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
aapl_prices['Date'] = pd.to_numeric(aapl_prices['Date'].str.replace('-',''))
rate['Date'] = pd.to_datetime(rate['Date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
rate['Date'] = pd.to_numeric(rate['Date'].str.replace('-',''))

#merge rate with df on date, using forward/backwards fill approach, as some dates of options and treasury prices do not line up...
rate_reindexed = rate.set_index('Date').reindex(df['date'])
rate_reindexed = rate_reindexed.ffill().bfill()
df = pd.merge(df, rate_reindexed, left_on='date', right_index=True, how='left')

#choose risk free rate 'r', based on which treasury rate matures closest to the expiration date of the option.
#Then drop other treasury columns leaving just 'r'
def determine_r(row):
    if row['days_expiry'] < 45:
        return row['1 Mo']
    elif 45 <= row['days_expiry'] < 75:
        return row['2 Mo']
    else:
        return row['3 Mo']

df['r'] = df.apply(determine_r, axis=1)
df = df.drop(['1 Mo', '2 Mo', '3 Mo', '6 Mo', '1 Yr'], axis=1)

#Now to the price dataframe...

#create df with n_prices + 1 columns. first column indicating the date on the last pricing. other n_prices columns will be the n_prices leading up to the current date.
#then we will merge again on date, using most recent closing price preceding option pricing.
n = 10
#obtain df of all prices needed for last_n_prices df... and convert closing price to decimal type
minDateIndex = aapl_prices.index[aapl_prices['Date'] == min(df['date'])-1].tolist()[0]
maxDateIndex = aapl_prices.index[aapl_prices['Date'] == max(df['date'])-1].tolist()[0]
prices_all = aapl_prices.loc[maxDateIndex:(minDateIndex+n),].sort_values(by=['Date'])
prices_all[' Close/Last'] = prices_all[' Close/Last'].str.replace('$', '').astype(float)
prices_all = prices_all.drop([' Volume', ' Open', ' High', ' Low'], axis=1)

stepData = []
for i in range(len(prices_all) - n):
    date = prices_all['Date'].iloc[i + n] 
    n_prices = prices_all[' Close/Last'].iloc[i:i + n + 1].tolist() 
    stepData.append([date] + n_prices)
columns = ['Date'] + [f't{i+1}' for i in range(n)] + ['currentP']
lastn_prices = pd.DataFrame(stepData, columns=columns)

df = pd.merge_asof(df, lastn_prices, left_on='date', right_on='Date', direction='backward')

#Create column for 'fair price' of option, just average of bid and ask... then drop out the rows not being used in this first iteration...
df['option_fp'] = (df['ask'] + df['bid'])/2
inputs_df = df.drop(['date', 'act_symbol', 'bid', 'ask', 'Date'], axis=1)


/var/folders/49/x210g1j14bs717pp0q4xf3zr0000gq/T/ipykernel_48050/4159526863.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['days_expiry'] = (pd.to_datetime(df['expiration']) - pd.to_datetime(df['date'])).dt.days


**Preparing Train/Test Data**

In [14]:
from sklearn.model_selection import train_test_split

#y is option_fp
#train and test calls and puts seperately (obviously)
display(inputs_df.head())

call_df = inputs_df[inputs_df['call_put'] == "Call"]
call_df = call_df.drop(['call_put'], axis=1)
put_df = inputs_df[inputs_df['call_put'] == "Put"]
put_df = put_df.drop(['call_put'], axis=1)

display(call_df.head())
CALL_X_train, CALL_X_test, CALL_Y_train, CALL_Y_test = train_test_split(call_df.drop(['option_fp'], axis=1).values, call_df['option_fp'].values, 
                                                                        test_size=.1, random_state=1)

PUT_X_train, PUT_X_test, PUT_Y_train, PUT_Y_test = train_test_split(put_df.drop(['option_fp'], axis=1).values, put_df['option_fp'].values, 
                                                                        test_size=.1, random_state=1)

#for input to LSTM-MLP, must split inputs into state inputs (for LSTM), and non-state inputs. We can pass the state inputs seperately through the
#LSTM, then take the LSTM output and concatenate it with the remaining non-state inputs. Thus we must split the inputs into two list elements,
#the first of which being a list of state-inputs (all price data), the second of which being a list of the remaining inputs.

#Note... Should we put the current asset price as part of the state inputs, or the non-state inputs?? try both....
#let's start with putting all pricing info in the state:
#dropping volatility from the LSTM-MLP inputs... Need to keep it in train/test split s.t. we can test the error of BSM using same y data.
BSM_CALL_X_train, BSM_CALL_X_test, BSM_CALL_Y_train, BSM_CALL_Y_test = CALL_X_train, CALL_X_test, CALL_Y_train, CALL_Y_test

LSTM_CALL_X_train, LSTM_CALL_X_test, LSTM_CALL_Y_train, LSTM_CALL_Y_test = np.delete(CALL_X_train,1,1), np.delete(CALL_X_test,1,1), CALL_Y_train, CALL_Y_test
LSTM_PUT_X_train, LSTM_PUT_X_test, LSTM_PUT_Y_train, LSTM_PUT_Y_test = np.delete(PUT_X_train,1,1), np.delete(PUT_X_test,1,1), PUT_Y_train, PUT_Y_test

LSTM_CALL_X_train = [LSTM_CALL_X_train[:,3:].reshape(LSTM_CALL_X_train.shape[0],n+1,1), LSTM_CALL_X_train[:,:3]]
LSTM_CALL_X_test = [LSTM_CALL_X_test[:,3:].reshape(LSTM_CALL_X_test.shape[0],n+1,1), LSTM_CALL_X_test[:,:3]]
LSTM_PUT_X_train = [LSTM_PUT_X_train[:,3:].reshape(LSTM_PUT_X_train.shape[0],n+1,1), LSTM_PUT_X_train[:,:3]]
LSTM_PUT_X_test = [LSTM_PUT_X_test[:,3:].reshape(LSTM_PUT_X_test.shape[0],n+1,1), LSTM_PUT_X_test[:,:3]]

,strike,call_put,vol,days_expiry,r,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,currentP,option_fp
0,145.0,Call,0.4236,13,2.42,157.76,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,170.41,25.675
1,145.0,Call,0.4236,13,2.42,157.76,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,170.41,25.675
2,145.0,Call,0.4236,13,2.42,157.76,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,170.41,25.675
3,145.0,Call,0.4236,13,2.42,157.76,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,170.41,25.675
4,145.0,Call,0.4236,13,2.42,157.76,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,170.41,25.675


,strike,vol,days_expiry,r,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,currentP,option_fp
0,145.0,0.4236,13,2.42,157.76,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,170.41,25.675
1,145.0,0.4236,13,2.42,157.76,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,170.41,25.675
2,145.0,0.4236,13,2.42,157.76,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,170.41,25.675
3,145.0,0.4236,13,2.42,157.76,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,170.41,25.675
4,145.0,0.4236,13,2.42,157.76,156.3,154.68,165.25,166.44,166.52,171.25,174.18,174.24,170.94,170.41,25.675


**First benchmarking error on Black-Scholes Model**

BSM Implementation:

In [44]:
from scipy.stats import norm

N = norm.cdf

def BS_CALL(params: np.array):
    K = params[0]
    sigma = params[1]
    T = params[2]/365
    r = params[3]/100
    S = params[-1]
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

def BS_PUT(params: np.array):
    K = params[0]
    sigma = params[1]
    T = params[2]/365
    r = params[3]/100
    S = params[-1]
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)

BSM Benchmark Error:

In [46]:
#need following columns in this order:
#CurrentP (S) [15] AKA [-1], strike (K) [0], days_expiry (T) [2], r (r) [3], vol (sigma) [1]

#then calculate price for all rows of test sets, then calculate squared error for each price, take mean, 
#compare with mse from one of the LSTM models on the test set (to-do)

BS_CALL_res = [BS_CALL(elem) for elem in CALL_X_test]
BS_PUT_res = [BS_PUT(elem) for elem in PUT_X_test]

BS_CALL_mse = np.mean((BS_CALL_res - CALL_Y_test)**2)
BS_PUT_mse = np.mean((BS_PUT_res - PUT_Y_test)**2)
print(BS_CALL_mse)
print(BS_PUT_mse)

print(CALL_X_test[0], CALL_Y_test[0])

0.028180376721682294
0.10644548580248238
[222.5      0.2283  48.       2.43   212.46   202.64   206.49   204.16
 205.53   209.01   208.74   205.7    209.19   213.28   213.26  ] 3.75


**Model Params**

In [47]:
layers = 4
features = 3
n_batch = 4096
n_epochs = 100

**Building LSTM-MLP Model**

In [48]:
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, LeakyReLU, BatchNormalization, LSTM, Bidirectional, Input, Concatenate
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
from keras.utils import plot_model

def make_model():
    close_history = Input((n+1, 1))
    input2 = Input((features,))
    
    lstm = Sequential()
    lstm.add(LSTM(units=8, input_shape=(n+1, 1), return_sequences=True))
    lstm.add(LSTM(units=8, return_sequences=True))
    lstm.add(LSTM(units=8, return_sequences=True))
    lstm.add(LSTM(units=8, return_sequences=False))
    input1 = lstm(close_history)
    
    connect = Concatenate()([input1, input2])
    
    for _ in range(layers - 1):
        connect = Dense(100)(connect)
        connect = BatchNormalization()(connect)
        connect = LeakyReLU()(connect)
    
    predict = Dense(1, activation='relu')(connect)

    return Model(inputs=[close_history, input2], outputs=predict)

2024-08-23 09:34:48.097323: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [49]:
call_model = make_model()
call_model.summary()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 11, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 8)         │      1,952 │ input_layer[0][0] │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 11)        │          0 │ sequential[0][0], │
│ (Concatenate)       │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 100)       │      1,200 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 100)       │        400 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu         │ (None, 100)       │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 100)       │     10,100 │ leaky_re_lu[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 100)       │        400 │ dense_1[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_1       │ (None, 100)       │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 100)       │     10,100 │ leaky_re_lu_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 100)       │        400 │ dense_2[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_2       │ (None, 100)       │          0 │ batch_normalizat… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │        101 │ leaky_re_lu_2[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 24,653 (96.30 KB)

 Trainable params: 24,053 (93.96 KB)

 Non-trainable params: 600 (2.34 KB)

**Testing a few parameters in base LSTM-MLP model**

In [50]:
call_model.compile(optimizer=Adam(learning_rate=1e-2), loss='mse')
history = call_model.fit(LSTM_CALL_X_train, LSTM_CALL_Y_train, 
                    batch_size=n_batch, epochs=10, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)
#call_model.save('saved-models/call_test1.keras')

Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 146ms/step - loss: 88.1718 - val_loss: 170.3651
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - loss: 45.8950 - val_loss: 170.3651
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - loss: 43.4663 - val_loss: 170.3651
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 42.6351 - val_loss: 170.3651
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 42.5904 - val_loss: 170.3651
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 42.3579 - val_loss: 170.3651
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - loss: 41.7564 - val_loss: 170.3651
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 41.6887 - val_loss: 170.3651
Epoch 9/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - loss: 41.7781 - val_loss: 166.6733
Epoch 10/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - loss: 41.5675 - val_loss: 155.6862


In [51]:
call_model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse')
history = call_model.fit(LSTM_CALL_X_train, LSTM_CALL_Y_train, 
                    batch_size=n_batch, epochs=5, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)
#call_model.save('saved-models/call_test2.keras')

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 122ms/step - loss: 41.4780 - val_loss: 146.1733
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - loss: 41.4493 - val_loss: 120.2511
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 41.0813 - val_loss: 98.3655
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 41.5924 - val_loss: 68.9267
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - loss: 41.5057 - val_loss: 56.6855


In [52]:
call_model.compile(optimizer=Adam(learning_rate=1e-4), loss='mse')
history = call_model.fit(LSTM_CALL_X_train, LSTM_CALL_Y_train, 
                    batch_size=n_batch, epochs=5, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)
#call_model.save('saved-models/call_test3.keras')

Epoch 1/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 139ms/step - loss: 40.7282 - val_loss: 52.1281
Epoch 2/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 40.6747 - val_loss: 48.4194
Epoch 3/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - loss: 41.0251 - val_loss: 46.0907
Epoch 4/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - loss: 41.1576 - val_loss: 44.6325
Epoch 5/5
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - loss: 41.1026 - val_loss: 43.7217
